In [1]:
import h5py
import numpy as np
import random
import pdb
from generator_helper_functions import create_user_maxtrix, create_user_ids, latent_user_input_layer

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = np.array([dset[i] for i in train_indicies])
validation_data = np.array([dset[i] for i in validation_indicies])
print("Training and Validation datasets separated")

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


FileNotFoundError: [Errno 2] No such file or directory: 'data/name_one_hot_dict.pickle'

In [ ]:
user_ids = create_user_ids(dset)
k = 50

In [ ]:
del dset_indicies
del train_indicies
del validation_indicies
del dset

In [ ]:
import pickle

In [ ]:
with open('data/user_matrix.pickle', 'wb') as handle:
    pickle.dump(user_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/training_data.pickle', 'wb') as handle:
    pickle.dump(training_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/validation_data.pickle', 'wb') as handle:
    pickle.dump(validation_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/user_ids.pickle', 'wb') as handle:
    pickle.dump(user_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
del user_matrix
del training_data
del validation_data
del user_ids

In [1]:
import pickle
with open('data/user_matrix.pickle', 'rb') as handle:
    user_matrix = pickle.load(handle)
with open('data/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)
with open('data/validation_data.pickle', 'rb') as handle:
    validation_data = pickle.load(handle)
with open('data/user_ids.pickle', 'rb') as handle:
    user_ids = pickle.load(handle)

In [2]:
from sklearn.preprocessing import MinMaxScaler

In [3]:
transformer = MinMaxScaler(copy=False)
transformer.fit_transform(user_matrix)
user_matrix

array([[0.45480946, 0.5649543 , 0.54005206, ..., 0.78012514, 0.7245458 ,
        0.97671676],
       [0.52922237, 0.48295796, 0.55065936, ..., 0.8655711 , 0.40475172,
        0.66800594],
       [0.5129979 , 0.57189465, 0.4832853 , ..., 0.772317  , 0.7031396 ,
        0.74582773],
       ...,
       [0.5170382 , 0.5100255 , 0.53248763, ..., 0.7930147 , 0.7573515 ,
        0.9455885 ],
       [0.47609553, 0.5598326 , 0.5154664 , ..., 0.770388  , 0.6871689 ,
        0.9969276 ],
       [0.46394268, 0.54970384, 0.49299762, ..., 0.7700641 , 0.6939482 ,
        0.98722965]], dtype=float32)

In [4]:
k = 50      
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
#import random
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import  latent_user_input_layer, Generate_User_Latent_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()


# k dim user vector + number of movies + statistics + month, day, year
input_size = k + 17770

batch_size = 64

model = Sequential()
model.add(Dense(input_size, activation='relu', input_shape=(input_size,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_logarithmic_error',
             optimizer='sgd',
             metrics=['mse', 'mae', 'acc'])

print("Model Compiled")

Using TensorFlow backend.


Model Compiled


In [ ]:
k = 50        
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
#import random
import numpy as np
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import  latent_user_input_layer, Generate_User_Latent_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()


# k dim user vector + number of movies + statistics + month, day, year
input_size = k + 17770

batch_size = 64

model = Sequential()
model.add(Dense(input_size//2, activation='relu', input_shape=(input_size,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mae',
             optimizer='sgd',
             metrics=['mse', 'acc'])

print("Model Compiled")

In [ ]:
input_size

In [ ]:
train_gen = Generate_User_Latent_Feedforward(training_data, 
                                batch_size, 
                                (input_size,), 
                                (1,), #label shape
                                latent_user_input_layer, 
                                lambda x: x[2],
                                user_matrix,
                                user_ids,
                                normalized = True
                                )

validation_gen = Generate_User_Latent_Feedforward(validation_data, 
                                batch_size, 
                                (input_size,), 
                                (1,), #label shape
                                latent_user_input_layer, 
                                lambda x: x[2],
                                user_matrix,
                                user_ids,
                                normalized = True
                                )

checkpointer = ModelCheckpoint(filepath='models/latent_user_SGD_MSLE_K50_v1__normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=2,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

Epoch 1/2
1000/1000 [==============================] - 1905s 2s/step - loss: 0.0852 - mean_squared_error: 1.3194 - mean_absolute_error: 0.9585 - acc: 0.2945 - val_loss: 0.0754 - val_mean_squared_error: 1.2120 - val_mean_absolute_error: 0.9308 - val_acc: 0.2992

Epoch 00001: val_loss improved from inf to 0.07542, saving model to models/latent_user_SGD_MSLE_K50_v1__normalized.hdf5
Epoch 2/2
 393/1000 [==========>...................] - ETA: 18:21 - loss: 0.0736 - mean_squared_error: 1.2052 - mean_absolute_error: 0.9291 - acc: 0.3034

In [6]:
validation_data[1]

array([   12238,   571810,        4, 20051206])

In [ ]:
from sklearn.metrics import precision_score

In [ ]:
precision_score(
    list(map(lambda x: round(float(model.predict(np.expand_dims(latent_user_input_layer(x, user_matrix, user_ids, True), axis=0)) * 5)), 
             validation_data[0:3000])), 
    validation_data[0:3000,2], average='micro') 



In [ ]:
validation_data[0:3, 2]

In [ ]:
user_ids = {user:i for i, user in enumerate(user_ids)}

In [ ]:
user_rating_matrix

In [ ]:
user_rating_matrix